# Chapter 2 — An Array of Sequences

**Sections with code snippets in this chapter:**

* [List Comprehensions and Generator Expressions](#List-Comprehensions-and-Generator-Expressions)
* [Tuples Are Not Just Immutable Lists](#Tuples-Are-Not-Just-Immutable-Lists)
* [Unpacking sequences and iterables](#Unpacking-sequences-and-iterables)
* [Pattern Matching with Sequences](#Pattern-Matching-with-Sequences)
* [Slicing](#Slicing)
* [Using + and * with Sequences](#Using-+-and-*-with-Sequences)
* [Augmented Assignment with Sequences](#Augmented-Assignment-with-Sequences)
* [list.sort and the sorted Built-In Function](#list.sort-and-the-sorted-Built-In-Function)
* [When a List Is Not the Answer](#When-a-List-Is-Not-the-Answer)
* [Memory Views](#Memory-Views)
* [NumPy and SciPy](#NumPy-and-SciPy)
* [Deques and Other Queues](#Deques-and-Other-Queues)
* [Soapbox](#Soapbox)

## List Comprehensions and Generator Expressions

#### Example 2-1. Building a list of Unicode codepoints from a string

In [1]:
symbols = '$¢£¥€¤'
codes = []

for symbol in symbols:
    codes.append(ord(symbol))

codes

[36, 162, 163, 165, 8364, 164]

#### Example 2-2. Build a list of Unicode codepoints from a string, using a listcomp

In [3]:
symbols = '$¢£¥€¤'

codes = [ord(symbol) for symbol in symbols]

codes

[36, 162, 163, 165, 8364, 164]

#### Box: Listcomps No Longer Leak Their Variables

In [4]:
x = 'ABC'
codes = [ord(x) for x in x]
x

'ABC'

In [5]:
codes

[65, 66, 67]

In [6]:
codes = [last := ord(c) for c in x]
last

67

#### Example 2-3. The same list built by a listcomp and a map/filter composition

In [8]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

[162, 163, 165, 8364, 164]

In [9]:
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
beyond_ascii

[162, 163, 165, 8364, 164]

#### Example 2-4. Cartesian product using a list comprehension

In [10]:
colors  = ['black', 'white']
sizes   = ['S', 'M', 'L']
tshirts =  [(color, size) for color in colors for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

In [12]:
for color in colors:
    for size in sizes:
        print((color, size))

('black', 'S')
('black', 'M')
('black', 'L')
('white', 'S')
('white', 'M')
('white', 'L')


In [17]:
shirts = [(color, size) for size in sizes for color in colors]
tshirts

[('black', 'S'),
 ('white', 'S'),
 ('black', 'M'),
 ('white', 'M'),
 ('black', 'L'),
 ('white', 'L')]

#### Example 2-5. Initializing a tuple and an array from a generator expression

In [18]:
symbols = '$¢£¥€¤'
tuple(ord(symbol) for symbol in symbols)

(36, 162, 163, 165, 8364, 164)

In [19]:
import array

array.array('I', (ord(symbol) for symbol in symbols))

array('I', [36, 162, 163, 165, 8364, 164])

#### Example 2-6. Cartesian product in a generator expression

In [21]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']

for tshirt in ('%s %s' % (c, s) for c in colors for s in sizes):
    print(tshirt)

black S
black M
black L
white S
white M
white L


## Tuples Are Not Just Immutable Lists

#### Example 2-7. Tuples used as records

In [22]:
nbo_coordinates = (-1.2833300, 36.8166700)
city, year, pop, chg, area = ('Nairobi', 2023, 5_325, 0.78, 703.9)
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]

for passport in sorted(traveler_ids):
    print('%s/%s' % passport)

BRA/CE342567
ESP/XDA205856
USA/31195855


In [23]:
for country, _ in traveler_ids:
    print(country)

USA
BRA
ESP


### Tuples as Immutable Lists

In [24]:
a = (10, 'alpha', [1, 2])
b = (10, 'alpha', [1, 2])

a == b

True

In [28]:
b[-1].append(199)
a == b

False

In [29]:
b

(10, 'alpha', [1, 2, 199, 199, 199])

In [31]:
def fixed(o):
    try:
        hash(o)
    except TypeError:
        return False
    return True

tf = (10, 'alpha', (1, 2))      # contains no mutable items
tm = (10, 'alpha', [1, 2])      # contains a mutable item(list)
fixed(tf)

True

In [32]:
fixed(tm)

False

## Unpacking sequences and iterables

In [ ]:
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates  # unpacking
latitude

In [ ]:
longitude

In [ ]:
divmod(20, 8)

In [ ]:
t = (20, 8)
divmod(*t)

In [ ]:
quotient, remainder = divmod(*t)
quotient, remainder

In [ ]:
import os

_, filename = os.path.split('/home/luciano/.ssh/id_rsa.pub')
filename

### Using * to grab excess items

In [1]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [2]:
a, b, *rest = range(3)
a, b, rest

(0, 1, [2])

In [3]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

In [4]:
a, *body, c, d = range(5)
a, body, c, d

(0, [1, 2], 3, 4)

In [5]:
*head, b, c, d = range(5)
head, b, c, d

([0, 1], 2, 3, 4)

### Unpacking with * in function calls and sequence literals

In [1]:
def fun(a, b, c, d, *rest):
    return a, b, c, d, rest


fun(*[1, 2], 3, *range(4, 7))

(1, 2, 3, 4, (5, 6))

In [2]:
*range(4), 4

(0, 1, 2, 3, 4)

In [3]:
[*range(4), 4]

[0, 1, 2, 3, 4]

In [4]:
{*range(4), 4, *(5, 6, 7)}

{0, 1, 2, 3, 4, 5, 6, 7}

### Nested unpacking
#### Example 2-8. Unpacking nested tuples to access the longitude

[02-array-seq/metro_lat_lon.py](02-array-seq/metro_lat_lon.py)

#### Example 2-12. Pattern matching with match/case—requires Python ≥ 3.10.
[02-array-seq/lispy/py3.10/lis.py](02-array-seq/lispy/py3.10/lis.py)

## Slicing

### Why Slices and Range Exclude the Last Item

In [1]:
l = [10, 20, 30, 40, 50, 60]

l[:2]  # split at 2

[10, 20]

In [2]:
l[2:]

[30, 40, 50, 60]

In [3]:
l[:3]  # split at 3

[10, 20, 30]

In [4]:
l[3:]

[40, 50, 60]

### Slice Objects

In [5]:
s = 'bicycle'
s[::3]

'bye'

In [6]:
s[::-1]

'elcycib'

In [7]:
s[::-2]

'eccb'

#### Example 2-13. Line items from a flat-file invoice

In [8]:
invoice = """
0.....6.................................40........52...55........
1909 Pimoroni PiBrella                      $17.50    3    $52.50
1489 6mm Tactile Switch x20                  $4.95    2    $9.90
1510 Panavise Jr. - PV-201                  $28.00    1    $28.00
1601 PiTFT Mini Kit 320x240                 $34.95    1    $34.95
"""

SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)

line_items = invoice.split('\n')[2:]

for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])

    $17.50   imoroni PiBrella                  
     $4.95   mm Tactile Switch x20             
    $28.00   anavise Jr. - PV-201              
    $34.95   iTFT Mini Kit 320x240             
 


### Assigning to Slices

In [9]:
l = list(range(10))
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [10]:
l[2:5] = [20, 30]
l

[0, 1, 20, 30, 5, 6, 7, 8, 9]

In [11]:
del l[5:7]
l

[0, 1, 20, 30, 5, 8, 9]

In [12]:
l[3::2] = [11, 22]
l

[0, 1, 20, 11, 5, 22, 9]

By design, this example raises an exception::

In [13]:
try:
    l[2:5] = 100
except TypeError as e:
    print(repr(e))

TypeError('can only assign an iterable')


In [14]:
l[2:5] = [100]
l

[0, 1, 100, 22, 9]

## Using + and * with Sequences

In [15]:
l = [1, 2, 3]
l * 5

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

In [16]:
5 * 'abcd'

'abcdabcdabcdabcdabcd'

### Building Lists of Lists